##### ℹ️ Process: This document describes the process of re-creation, general challenges and work arounds that have been advised for it.



In [1]:
import pandas as pd

easy_path = '../data/input/ez/'
easy_output_path = '../data/output/ez/'

#### Part 0:

deflaker is the main data set has been used in this project. It monitors the followin datasets:
    


In [2]:
projects = pd.read_csv(easy_path + 'historical_projects.csv')

projects.head(10)

,Project Name,Project Address
0,Achilles,git@github.com:doanduyhai/Achilles.git
1,ambari,git@github.com:apache/ambari.git
2,assertj-core,git@github.com:joel-costigliola/assertj-core.git
3,checkstyle,git@github.com:checkstyle/checkstyle.git
4,cloudera.oryx,git@github.com:cloudera/oryx.git
5,commons-exec,git@github.com:apache/commons-exec.git
6,dropwizard,git@github.com:dropwizard/dropwizard.git
7,hadoop,git@github.com:apache/hadoop.git
8,handlebars.java,git@github.com:jknack/handlebars.java.git
9,hbase,git@github.com:apache/hbase.git



#### Part 1:

To detect flaky test samples, we have to run "mine_test_files.py" using "historical_rerun_flaky_tests.csv"

❌ Unfortunately, this code does not run. -> Solution: I had to fix the logic for this file.

In [3]:
projects = pd.read_csv(easy_path + 'historical_rerun_flaky_tests.csv')

projects.head(10)

,Project,sha,Test class,Test method
0,achilles,2a82e0a011197906a8471be487da8b24ffaa39d9,info.archinnov.achilles.it.TestEntityWithStati...,should_find_using_static_consistency
1,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestALLEntityAsChild,should_delete_by_id
2,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestALLEntityAsChild,should_dsl_delete
3,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestALLEntityAsChild,should_dsl_update_child_value
4,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestALLEntityAsChild,should_insert
5,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestCRUDSimpleEntity,should_delete_by_id
6,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestCRUDSimpleEntity,should_delete_by_partition
7,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestCRUDSimpleEntity,should_delete_with_equal_condition
8,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestCRUDSimpleEntity,should_delete_with_inequal_condition
9,achilles,3ef293744f913c1aa8cf59d2fdd78f46a5e2f79f,info.archinnov.achilles.it.TestCRUDSimpleEntity,should_delete_with_not_equal_condition


#### Part 2:

To detect non-flaky test sample, the paper relies on running each of the tests '100' time then based on the 
rereuns tags fully successful ones as LIKELY to be nonflaky.

❌ Unfortunately, the git repository does not provide the explination as to how did they ran the tests.
-> Solution -> I have completely re-made the logic using docker running Java.


### Part 3:

Raw Frequenices of each word within the falky test -> this is the starting point for flaky vocabulary investigations


In [4]:
projects = pd.read_csv(easy_output_path + 'features_raw.csv')

projects.head(15)

,token,count
0,assertequ,850
1,job,550
2,creat,492
3,add,471
4,servic,466
5,action,453
6,asserttru,426
7,conf,411
8,tabl,408
9,start,406


After applying filters the potential vocabulary of flakiness are:

In [5]:
projects = pd.read_csv(easy_output_path + 'features_frequency.csv')

projects.head(15)

,token,count
0,job,571
1,service,466
2,action,457
3,start,424
4,file,421
5,run,394
6,fail,369
7,execut,356
8,call,353
9,time,348


Part 3:

Issues regarding running Java:

It is never mentioned how exactly do they handle the parts that use java and the building the main jar files they use. To make it easy, I just made a docker that autoamtically builds and runs all those sub-parts:

Basically, we just need an Alpine or Linux that can run JDK with injected Gradle. Then I run multiple commands needed to build and run Java-related parts in the method : '__actuall.py :: separate_all_tests()__'



> for further you can refer to docker-compose.yml

Finally, the rest of the code run more or less smooth. I had to modify some parts of the code any part that worked fine; I just directly imported using '\_\_import()\_\_'  suite.

⚠️ This document might be subject of copy-right. All rights reserved for the orginial creators. -Amir Dashti